In [7]:
import numpy as np
import pandas as pd
import math
from sympy import primerange

from scipy.stats import entropy

# --- Constants ---
phi = (1 + math.sqrt(5)) / 2
s_values = [round(s, 2) for s in np.linspace(0.90, 1.10, 21)]
N_values = list(range(20, 101, 5))
EXPONENT_LIMIT = 700  # conservative threshold for log tracking

# --- Colab-safe symbolic log trace of ζ_{φ,s}(N) ---
def log_symbolic_trace(N, s):
    primes = list(primerange(1, 2000))[:N]
    trace_log_terms = []

    for i, p in enumerate(primes):
        exponent = phi**(i + 1) * s
        log_term = exponent * math.log(p)
        if log_term < EXPONENT_LIMIT:
            trace_log_terms.append(-log_term)
        else:
            trace_log_terms.append(float('-inf'))  # explicitly mark underflow

    return trace_log_terms

# --- φⁿ mod base folding ---
def phi_mod_cycle(n_terms, base):
    return [phi**i % base for i in range(1, n_terms + 1)]

# --- Digit entropy from significant digits of log terms ---
def digit_entropy(x_list, precision=10):
    all_digits = ""
    for x in x_list:
        if x == float('-inf'):
            continue
        digits = f"{abs(x):.{precision}f}".replace('.', '')[:precision]
        all_digits += digits
    if not all_digits:
        return 0.0
    freqs = [all_digits.count(str(d)) / len(all_digits) for d in range(10)]
    return entropy(freqs, base=2)

# --- Run suite ---
records = []

for s in s_values:
    logs_by_N = []
    entropies = []
    mod_residues = phi_mod_cycle(len(N_values), base=9)

    for N in N_values:
        log_terms = log_symbolic_trace(N, s)
        logs_by_N.append(log_terms)
        entropies.append(digit_entropy(log_terms, precision=10))

    records.append({
        "s": s,
        "mean_entropy": np.mean(entropies),
        "entropy_std": np.std(entropies),
        "phi_mod_9_mean": np.mean(mod_residues),
        "phi_mod_9_std": np.std(mod_residues)
    })

df_symbolic_trace = pd.DataFrame(records)

# For Colab: display the result cleanly
from IPython.display import display
display(df_symbolic_trace)



,s,mean_entropy,entropy_std,phi_mod_9_mean,phi_mod_9_std
0,0.90,3.266303,0.0,4.493063,2.468974
1,0.91,3.281581,0.0,4.493063,2.468974
2,0.92,3.249519,0.0,4.493063,2.468974
3,0.93,3.223919,0.0,4.493063,2.468974
4,0.94,3.285501,0.0,4.493063,2.468974
5,0.95,3.245943,0.0,4.493063,2.468974
6,0.96,3.313954,0.0,4.493063,2.468974
7,0.97,3.275587,0.0,4.493063,2.468974
8,0.98,3.277901,0.0,4.493063,2.468974
9,0.99,3.254166,0.0,4.493063,2.468974
